In [1]:
import argparse
import awkward
import os.path as osp
import os
import glob
import torch
import awkward as ak
import time
import uproot
import uproot3
import yaml
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.datasets import MNISTSuperpixels
from torch_geometric.data import DataListLoader, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import SplineConv, global_mean_pool, DataParallel, EdgeConv, GATConv,PNAConv
from torch_geometric.data import Data
import scipy.sparse as ss
from datetime import datetime, timedelta
from torch_geometric.utils import degree
import os.path

from tools.GNN_model_weight.models import *
from tools.GNN_model_weight.utils  import *

print("Libraries loaded!")

/home/jmsardain/.local/lib/python3.8/site-packages/awkward0/__init__.py:8: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(numpy.__version__) < distutils.version.LooseVersion("1.13.1"):


Libraries loaded!


In [ ]:
scale_factor = 14.475606  

for variation in range(31, 32):
    ## top tagging
    if variation==30:
        path_to_test_file = "/data/jmsardain/LJPTagger/FullSplittings/SplitForTopTagger/5_Signal_and_BKG/*test.root"
        path_to_outdir = "/data/jmsardain/LJPTagger/Models/TopTagger/Scores/"
        path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/TopTagger/LundNet/LundNet_e030_0.02869.pt"
        output_name = "LundNetScores"
        choose_model = "LundNet"
        learning_rate = 0.0005
        batch_size = 2048
        scale_factor = 1

    if variation==31:
        path_to_test_file = "/data/ravinascos/Lund_Plane_2024_graphs/TEST_2/*.root"
        path_to_outdir = "/data/ravinascos/LundNet/R_22_Top/"
        path_to_combined_ckpt = "/home/ravinascos/LundNet/ljptagger_R22/Models/DUMMY_01QCD_05Z/LundNet_Zprime_R22_train_1QCD_3Z_Stand_e-4_e040_0.22656.pt"
        output_name = "LundNetScores"
        choose_model = "LundNet"
        learning_rate = 0.0005
        batch_size = 2048
        scale_factor = 1


    
    files = glob.glob(path_to_test_file)

    #print ("files:",files)
    intreename = "FlatSubstructureJetTree"

    nentries_total = 0
    nentries_done = 0

    for file in files:
        nentries_total += uproot3.numentries(file, intreename)

    print("Evaluating on {} files with {} entries in total.".format(len(files), nentries_total))
    
    #Load tf keras model
    # jet_type = "Akt10RecoChargedJet" #track jets
    jet_type = "Akt10UFOJet" #UFO jets

    t_filestart = time.time()

    count_files = 0 
    graph_small_example = []
    
    for file in files:
        t_start = time.time()
        dataset = []
        print("Loading file",file)

        with uproot.open(file) as infile:
            tree = infile[intreename]

            count_files += 1
            dsids_test = tree["dsid"].array(library="np")
            if dsids_test[0] == 364702 :
                continue

            jet_pts_truth = ak.to_numpy(ak.flatten(tree["LRJ_pt"].array(library="ak")) )
            #ptweights = ak.to_numpy(ak.flatten(tree["LRJ_pt"].array(library="ak")) )
            ptweights = numpy.ones_like( ak.to_numpy(ak.flatten(tree["LRJ_pt"].array(library="ak")) ))
            #cut_pt = (jet_pts_truth > 350000) #& (ptweights < 1000000 )
            
            dsids = ak.to_numpy(ak.flatten(tree["LRJ_truthLabel"].array(library="ak")) )
            mcweights = ak.to_numpy(ak.flatten(tree["LRJ_pt"].array(library="ak")) )  #mcEventWeight
            NBHadrons = ak.to_numpy(ak.flatten(tree["LRJ_pt"].array(library="ak")) )
            
            parent1 =  ak.flatten(tree["jetLundIDParent1"].array(library="ak")) 
            parent2 = ak.flatten(tree["jetLundIDParent2"].array(library="ak")) 
            jet_pts = ak.to_numpy(ak.flatten(tree["LRJ_pt"].array(library="ak")) )
            jet_etas = ak.to_numpy(ak.flatten(tree["LRJ_eta"].array(library="ak")) )
            jet_phis = ak.to_numpy(ak.flatten(tree["LRJ_phi"].array(library="ak")) )
            jet_ms =  ak.to_numpy(ak.flatten(tree["LRJ_mass"].array(library="ak")))
            all_lund_zs = ak.flatten(tree["jetLundZ"].array(library="ak")) 
            all_lund_kts = ak.flatten(tree["jetLundKt"].array(library="ak")) 
            all_lund_drs = ak.flatten(tree["jetLundDeltaR"].array(library="ak"))
            N_tracks = ak.to_numpy(ak.flatten(tree["LRJ_Nconst_Charged"].array(library="ak")) )

            labels = dsids

            flat_weights = GetPtWeight_2( dsids, jet_pts, filename=config['data']['weights_file'], SF=config['data']['scale_factor'])

            dataset = create_train_dataset_fulld_new_Ntrk_pt_weight_file_test( dataset, graph_small_example , all_lund_zs, all_lund_kts, all_lund_drs, parent1, parent2, flat_weights, labels ,N_tracks,jet_pts, jet_ms  )
        #######################################################################################################################
        s_evt = 0
        events = 100
        print("Dataset created!")
        delta_t_fileax = time.time() - t_start
        print("Created dataset in {:.4f} seconds.".format(delta_t_fileax))

        test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        print ("dataset dataset size:", len(dataset))


        #EVALUATING
        #torch.save(model.state_dict(), path)

        if choose_model == "LundNet":
            model = LundNet()
            # model = LundNet_old()
        if choose_model == "GATNet":
            model = GATNet()
        if choose_model == "GINNet":
            model = GINNet()
        if choose_model == "EdgeGinNet":
            model = EdgeGinNet()
        if choose_model == "PNANet":
            model = PNANet()

        model.load_state_dict(torch.load(path_to_combined_ckpt))

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Usually gpu 4 worked best, it had the most memory available
        model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        #Predict scores
        y_pred = get_scores(test_loader, model, device)
        #print(y_pred)
        tagger_scores = y_pred[:,0]
        delta_t_pred = time.time() - t_filestart - delta_t_fileax
        print("Calculated predicitions in {:.4f} seconds,".format(delta_t_pred))
        
        #Save root files containing model scores
        filename = file.split("/")[-1]
        outfile_path = os.path.join(path_to_outdir, filename)

        # tagger_scores += [-1] * (len(dsids) - len(tagger_scores))
        tagger_scores = np.array(tagger_scores)
        # tagger_scores = np.pad(tagger_scores, (0, len(dsids) - len(tagger_scores)), 'constant', constant_values=(-1))
        print ("dsids",len(dsids),"mcweights",len(mcweights),"NBHadrons",len(NBHadrons),"tagger_scores",len(tagger_scores),"jet_pts",len(jet_pts),"jet_etas",len(jet_phis),"jet_phis",len(jet_phis),"jet_ms",len(jet_ms),"ptweights",len(ptweights))
        with uproot3.recreate("{}_score_{}.root".format(outfile_path, output_name)) as f:
            treename = "FlatSubstructureJetTree"
            #Declare branch data types
            f[treename] = uproot3.newtree({"EventInfo_mcChannelNumber": "int32",
                                          "EventInfo_mcEventWeight": "float32",
                                          "EventInfo_NBHadrons": "int32",   # I doubt saving the parents is necessary here
                                          "fjet_nnscore": "float32",        # which is why I didn't include them
                                          "fjet_pt": "float32",
                                          "fjet_eta": "float32",
                                          "fjet_phi": "float32",
                                          "fjet_m": "float32",
                                          "fjet_weight_pt": "float32", 
                                           # "truthjet_pt" : "float32",
                                           # "ungroomedtruthjet_pt" : "float32",
                                           # "ungroomedtruthjet_m" : "float32",
                                           # "ungroomedtruthjet_split12" : "float32",
                                          })

       
            #Save branches
            f[treename].extend({"EventInfo_mcChannelNumber": dsids,
                                "EventInfo_mcEventWeight": mcweights,
                                "EventInfo_NBHadrons": NBHadrons,
                                "fjet_nnscore": tagger_scores,
                                "fjet_pt": jet_pts,
                                "fjet_eta": jet_etas,
                                "fjet_phi": jet_phis,
                                "fjet_m": jet_ms,
                                "fjet_weight_pt": ptweights,
                                # "truthjet_pt" : truth_jetpt,
                                # "ungroomedtruthjet_pt" : truth_ungroomedjet_pt,
                                # "ungroomedtruthjet_m" : truth_ungroomedjet_m,
                                # "ungroomedtruthjet_split12" : truth_ungroomedjet_split12,
                                })

        delta_t_save = time.time() - t_start - delta_t_fileax - delta_t_pred
        print("Saved data in {:.4f} seconds.".format(delta_t_save))

        #nentries = 0
        #Time statistics
        nentries_done += uproot3.numentries(file, intreename)
        time_per_entry = (time.time() - t_start)/nentries_done
        eta = time_per_entry * (nentries_total - nentries_done)

        print("Evaluated on {} out of {} events".format(nentries_done, nentries_total))
        print("Estimated time until completion: {}".format(str(timedelta(seconds=eta))))


    print("Total evaluation time: {:.4f} seconds.".format(time.time()-t_filestart))



Evaluating on 54 files with 3306709 entries in total.
Loading file /data/jmsardain/LJPTagger/FullSplittings/SplitForTopTagger/5_Signal_and_BKG/user.rvinasco.30617190._000001.tree.root_test.root


/home/jmsardain/LJPTagger/ljptagger/src/tools/GNN_model_weight/utils.py:377: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(edge).detach(),


Dataset created!
Created dataset in 4.4803 seconds.
dataset dataset size: 21800


/home/jmsardain/.local/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/home/jmsardain/.local/lib/python3.8/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Calculated predicitions in 5.8253 seconds,
dsids 21800 mcweights 21800 NBHadrons 21800 tagger_scores 21800 jet_pts 21800 jet_etas 21800 jet_phis 21800 jet_ms 21800 ptweights 21800
Saved data in 0.0913 seconds.
Evaluated on 21800 out of 3306709 events
Estimated time until completion: 0:26:06.972274
Loading file /data/jmsardain/LJPTagger/FullSplittings/SplitForTopTagger/5_Signal_and_BKG/user.jzahredd.37503560._000003.tree.root_test.root
Dataset created!
Created dataset in 67.6423 seconds.
dataset dataset size: 162061
Calculated predicitions in 22.9022 seconds,
dsids 162061 mcweights 162061 NBHadrons 162061 tagger_scores 162061 jet_pts 162061 jet_etas 162061 jet_phis 162061 jet_ms 162061 ptweights 162061
Saved data in -10.2379 seconds.
Evaluated on 183861 out of 3306709 events
Estimated time until completion: 0:22:44.018995
Loading file /data/jmsardain/LJPTagger/FullSplittings/SplitForTopTagger/5_Signal_and_BKG/user.rvinasco.30617190._000002.tree.root_test.root
Dataset created!
Created da

In [ ]:
# #data
# path_to_test_file = "/data/jmsardain/LJPTagger/Train_Test_split_lxplus/*_test.root"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/FinerLRadv/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/FinerLRadvBigger/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/NewLossCalc/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/AdvLR0p005LW0p1_1/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/AdvLR0p005LW0p001_1/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/AdvLR0p005LW0p0005_1/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/AdvLR0p0005LW0p0005_1/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/Loss2Lambda2AdvLR0p0005LW0p0005_1/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/Loss2Lambda10AdvLR0p002LW0p0005_1/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/Loss0p5Lambda0p3AdvLR0p001LW0p005_1/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/Loss0p5Lambda0p3AdvLR0p001LW0p05_1/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/Loss1Lambda0p2AdvLR0p001LW0p005_1/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/Loss1Lambda0p5AdvLR0p001LW0p005_1/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/Loss5Lambda10AdvLR0p005LW0p0005_1/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/Loss10Lambda10AdvLR0p005LW0p0002_1/"
# #path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/Loss10Lambda10AdvLR0p005LW0p001_1/"
# path_to_outdir = "/data/jmsardain/LJPTagger/Scores/LundNetWeighted/Loss10Lambda10AdvLR0p005LW0p0001_1/"

# #test
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/combined_class_lambda10_lossp10_lrr1_s50e100_-1279.70129_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/FinerLRadv/combined_class_lambda10_lossp10_lrr0.1_s50e008_-641.31677_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/NewLossCalc/combined_class_lambda10_lossp10_lrr10_s50e026_-809.88887_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/AdvLR0p005LW0p1_1/combined_class_lambda10_lossp10_lrr10_s50e075_-809.84754_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/AdvLR0p005LW0p001_1/combined_class_lambda10_lossp10_lrr10_s50e024_-809.88712_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/AdvLR0p005LW0p0005_1/combined_class_lambda10_lossp10_lrr10_s50e019_-1193.18775_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/AdvLR0p0005LW0p0005_1/combined_class_lambda10_lossp10_lrr1_s50e011_-1212.60889_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/Loss2Lambda2AdvLR0p0005LW0p0005_1/combined_class_lambda2_lossp2_lrr1_s50e018_-161.97569_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/Lambda2AdvLR0p0005LW0p0005_1/combined_class_lambda2_lossp10_lrr1_s50e027_-809.87599_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/Loss2Lambda10AdvLR0p002LW0p0005_1/combined_class_lambda10_lossp2_lrr4_s50e029_-161.97719_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/Loss0p5Lambda0p3AdvLR0p001LW0p005_1/combined_class_lambda0.3_lossp0.5_lrr2_s50e102_-40.49244_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/Loss0p5Lambda0p3AdvLR0p001LW0p05_1/combined_class_lambda0.3_lossp0.5_lrr2_s50e031_-40.44051_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/Loss0p5Lambda0p3AdvLR0p001LW0p05_1/combined_class_lambda0.3_lossp0.5_lrr2_s50e047_-40.47020_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/Loss1Lambda0p2AdvLR0p001LW0p005_1/combined_class_lambda0.2_lossp1_lrr2_s50e114_-124.13304_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/Loss1Lambda0p5AdvLR0p001LW0p005_1/combined_class_lambda0.5_lossp1_lrr2_s50e088_-80.98684_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/Loss5Lambda10AdvLR0p005LW0p0005_1/combined_class_lambda10_lossp5_lrr10_s50e026_-404.94411_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/Loss10Lambda10AdvLR0p005LW0p0002_1/combined_class_lambda10_lossp10_lrr10_s50e045_-809.88875_comb_.pt"
# #path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/Loss10Lambda10AdvLR0p005LW0p001_1/combined_class_lambda10_lossp10_lrr10_s50e017_-809.87103_comb_.pt"
# path_to_combined_ckpt = "/data/jmsardain/LJPTagger/Models/LundNetWeighted/Loss10Lambda10AdvLR0p005LW0p0001_1/combined_class_lambda10_lossp10_lrr10_s50e030_-809.88856_comb_.pt"
# output_name = "LundNetScores"
# choose_model = "LundNet"
# learning_rate = 0.0005
# batch_size = 2048

In [ ]:
# files = glob.glob(path_to_test_file)

# #print ("files:",files)
# intreename = "FlatSubstructureJetTree"

# nentries_total = 0
# nentries_done = 0

# for file in files:
#     nentries_total += uproot3.numentries(file, intreename)

# print("Evaluating on {} files with {} entries in total.".format(len(files), nentries_total))

In [ ]:
# #Load tf keras model
# # jet_type = "Akt10RecoChargedJet" #track jets
# jet_type = "Akt10UFOJet" #UFO jets

# t_filestart = time.time()

# for file in files:

#     t_start = time.time()

#     dsids = np.array([])
#     NBHadrons = np.array([])
#     mcweights = np.array([])
#     ptweights = np.array([])

#     all_lund_zs = np.array([])
#     all_lund_kts = np.array([])
#     all_lund_drs = np.array([])

#     parent1 = np.array([])
#     parent2 = np.array([])

#     jet_pts = np.array([])
#     jet_phis = np.array([])
#     jet_etas = np.array([])
#     jet_ms = np.array([])
#     eta = np.array([])
#     jet_truth_pts = np.array([])
#     jet_truth_etas = np.array([])
#     jet_truth_dRmatched = np.array([])
#     jet_truth_split = np.array([])
#     jet_ungroomed_ms = np.array([])
#     jet_ungroomed_pts = np.array([])
#     vector = []

#     print("Loading file",file)

#     with uproot.open(file) as infile:

#         tree = infile[intreename]
#         dsids = np.append( dsids, np.array(tree["DSID"].array()) )
#         #eta = ak.concatenate(eta, pad_ak3(tree["Akt10TruthJet_jetEta"].array(), 30),axis=0)
#         mcweights = np.append( mcweights, np.array(tree["mcWeight"].array()) )
#         ptweights = np.append( ptweights, np.array(tree["fjet_testing_weight_pt"].array()) )
#         NBHadrons = np.append( NBHadrons, ak.to_numpy(tree["Akt10UFOJet_GhostBHadronsFinalCount"].array()))

#         parent1 = np.append(parent1, tree["UFO_edge1"].array(library="np"),axis=0)
#         parent2 = np.append(parent2, tree["UFO_edge2"].array(library="np"),axis=0)

#         #Get jet kinematics
#         jet_pts = np.append(jet_pts, tree["UFOSD_jetPt"].array(library="np"))
#         jet_etas = np.append(jet_etas, tree["UFOSD_jetEta"].array(library="np"))
#         jet_phis = np.append(jet_phis, tree["UFOSD_jetPhi"].array(library="np"))
#         jet_ms = np.append(jet_ms, tree["UFOSD_jetM"].array(library="np"))

# #        jet_truth_pts = np.append(jet_truth_pts, tree["Truth_jetPt"].array(library="np"))
# #        jet_truth_etas = np.append(jet_truth_etas, ak.to_numpy(tree["Truth_jetEta"].array(library="np")))


#         #Get Lund variables
#         all_lund_zs = np.append(all_lund_zs,tree["UFO_jetLundz"].array(library="np") )
#         all_lund_kts = np.append(all_lund_kts, tree["UFO_jetLundKt"].array(library="np") )
#         all_lund_drs = np.append(all_lund_drs, tree["UFO_jetLundDeltaR"].array(library="np") )


#     #Get labels
#     labels = ( dsids > 370000 ) & ( NBHadrons == 0 )
#     labels = to_categorical(labels, 2)
#     labels = np.reshape(labels[:,1], (len(all_lund_zs), 1))

#     dataset = create_train_dataset_fulld_new(all_lund_zs, all_lund_kts, all_lund_drs, parent1, parent2, labels)
#     s_evt = 0
#     events = 100
#     print("Dataset created!")
#     delta_t_fileax = time.time() - t_start
#     print("Created dataset in {:.4f} seconds.".format(delta_t_fileax))

    
#     ## free memory
#     del all_lund_zs
#     del all_lund_kts
#     del all_lund_drs
#     del parent1
#     del parent2
#     del eta
#     del jet_truth_pts
#     del jet_truth_etas
#     del jet_truth_dRmatched
#     del jet_truth_split
#     del jet_ungroomed_ms
#     del jet_ungroomed_pts
    

#     test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

#     print ("dataset dataset size:", len(dataset))


#     #EVALUATING
#     #torch.save(model.state_dict(), path)
  
#     if choose_model == "LundNet":
#         model = LundNet()
#     if choose_model == "GATNet":
#         model = GATNet()
#     if choose_model == "GINNet":
#         model = GINNet()
#     if choose_model == "EdgeGinNet":
#         model = EdgeGinNet()
#     if choose_model == "PNANet":
#         model = PNANet()

#     model.load_state_dict(torch.load(path_to_combined_ckpt))

#     device = torch.device('cuda') # Usually gpu 4 worked best, it had the most memory available
#     model.to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#     #Predict scores
#     y_pred = get_scores(test_loader, model, device)
#     #print(y_pred)
#     tagger_scores = y_pred[:,0]
#     delta_t_pred = time.time() - t_filestart - delta_t_fileax
#     print("Calculated predicitions in {:.4f} seconds,".format(delta_t_pred))

#     #Save root files containing model scores
#     filename = file.split("/")[-1]
#     outfile_path = os.path.join(path_to_outdir, filename)

#     print ("dsids",len(dsids),"mcweights",len(mcweights),"NBHadrons",len(NBHadrons),"tagger_scores",len(tagger_scores),"jet_pts",len(jet_pts),"jet_etas",len(jet_phis),"jet_phis",len(jet_phis),"jet_ms",len(jet_ms),"ptweights",len(ptweights))
#     with uproot3.recreate("{}_score_{}.root".format(outfile_path, output_name)) as f:

#         treename = "FlatSubstructureJetTree"

#         #Declare branch data types
#         f[treename] = uproot3.newtree({"EventInfo_mcChannelNumber": "int32",
#                                       "EventInfo_mcEventWeight": "float32",
#                                       "EventInfo_NBHadrons": "int32",   # I doubt saving the parents is necessary here
#                                       "fjet_nnscore": "float32",        # which is why I didn't include them
#                                       "fjet_pt": "float32",
#                                       "fjet_eta": "float32",
#                                       "fjet_phi": "float32",
#                                       "fjet_m": "float32",
#                                       "fjet_weight_pt": "float32"
#                                       })

#         #Save branches
#         f[treename].extend({"EventInfo_mcChannelNumber": dsids,
#                             "EventInfo_mcEventWeight": mcweights,
#                             "EventInfo_NBHadrons": NBHadrons,
#                             "fjet_nnscore": tagger_scores,
#                             "fjet_pt": jet_pts,
#                             "fjet_eta": jet_etas,
#                             "fjet_phi": jet_phis,
#                             "fjet_m": jet_ms,
#                             "fjet_weight_pt": ptweights,
#                             })

#     delta_t_save = time.time() - t_start - delta_t_fileax - delta_t_pred
#     print("Saved data in {:.4f} seconds.".format(delta_t_save))

#     #nentries = 0
#     #Time statistics
#     nentries_done += uproot3.numentries(file, intreename)
#     time_per_entry = (time.time() - t_start)/nentries_done
#     eta = time_per_entry * (nentries_total - nentries_done)

#     print("Evaluated on {} out of {} events".format(nentries_done, nentries_total))
#     print("Estimated time until completion: {}".format(str(timedelta(seconds=eta))))


# print("Total evaluation time: {:.4f} seconds.".format(time.time()-t_filestart))
